In [1]:
#!fsharp
#!value --from-file input.txt --name input

In [1]:
#!fsharp
#!share input --from value

type RangeSet =
  | Range of int * int
  | Union of RangeSet list

let rec contains i = function
  | Range (min, max) -> min <= i && i <= max
  | Union ranges -> List.exists (contains i) ranges

let containedIn range i = contains i range

let parseRule (line:string) =
  let [|fieldName; ranges|] = line.Split(": ")
  fieldName, Union [Range (int ranges.[0..1], int ranges.[3..5]); Range (int ranges.[10..12], int ranges.[14..16])]

let (rules, ticket, nearbyTickets) = 
  match input.Trim().Split("\n\n") with
  | [| rules; ticket; nearby |] -> 
    (
      rules.Split('\n') |> Array.map parseRule, 
      ticket.Split('\n').[1].Split(',') |> Array.map int, 
      nearby.Split('\n').[1..] |> Array.map (fun s -> s.Split(',') |> Array.map int)
    )
  | _ -> failwith "Bad Input"

let anyRuleRange = rules |> Seq.map snd |> Seq.toList |> Union

nearbyTickets
|> Seq.collect id
|> Seq.filter (fun n -> not (contains n anyRuleRange))
|> Seq.sum

28882

In [1]:
#!fsharp
let validTickets =
  nearbyTickets
  |> Array.filter (fun ticket -> ticket |> Seq.forall (containedIn anyRuleRange))

let matchingRules =
  validTickets
  |> Array.transpose 
  |> Array.map (fun vs -> 
    rules 
    |> Array.filter (fun r -> vs |> Seq.forall (containedIn (snd r))) 
    |> Array.map fst 
    |> set)
  |> Array.indexed
  |> set
  
let rec solve constraints =
  if constraints |> Set.isEmpty then Some Map.empty else
  let (i, possibleValues) as min = constraints |> Set.toSeq |> Seq.minBy (snd >> Set.count)
  if possibleValues |> Set.isEmpty then None else
  let remainingConstraints = constraints |> Set.remove min 
  possibleValues 
  |> Set.toSeq 
  |> Seq.tryPick (fun v -> 
    remainingConstraints
    |> Set.map (fun (i, s) -> (i, Set.remove v s))
    |> solve
    |> Option.map (Map.add i v)
  )

let result = solve matchingRules

result.Value
|> Map.filter (fun i s -> s.StartsWith("departure"))
|> Map.toSeq
|> Seq.map (fun (i,_) -> int64 ticket.[i])
|> Seq.reduce (*)


1429779530273